This is a quick script for generating a .pts input file for Pets2 from data acquired with instamatic. The script generates a .txt file with the appropriate formatting and assigns to each frame the correct tilt angle, tanking into account defocussed frames. 

In [8]:
import os

In [9]:
directory = 'E:/PhD/Pets/Fish/Fish_cRED_150cm_ARM/fish_150cm_2/tiff'

In [10]:
#some params
wavelength = 0.0251 
Aperpixel = 0.005413718984556342 #calibration factor
phi = 0 #precession angle
G_gamma = 1.1 #detector gain
Psi = 0.1 #variance of detector noise
reflectionsize = 12
binning = 1

Generating the .txt input file, without tilt information

In [ ]:
content = (
f"lambda {wavelength}\n"
"\n"
f"aperpixel {Aperpixel}\n"
"\n"
f"phi {phi}\n"
"\n"
f"noiseparameters {G_gamma} {Psi}\n"
"\n"
f"reflectionsize {reflectionsize}\n"
"\n"
f"bin {binning}"
"\n"
"\n"
"imagelist\n"
)
for filename in os.listdir(directory):
    if filename.endswith('.tiff'):
        content += f"tiff/{filename} 0 0\n"

content += 'endimagelist'

output_file = 'fish_cRED_manualinput.txt'

with open(output_file, 'w') as file:
    file.write(content)

Assigning to each frame the correct tilt angle

In [12]:
#Read the starting angle and oscillation range (tilt step) from the instamatic log file
logpath = 'E:/PhD/Pets/Fish/Fish_cRED_150cm_ARM/fish_150cm_2/cRED_log.txt'

starting_angle = None
ending_angle = None
oscillation_angle = None

with open(logpath) as file:
    for line in file:
        if "Starting angle" in line:
            starting_angle = float(line.split(":")[1].strip().split()[0])
        elif "Ending angle" in line:
            ending_angle = float(line.split(":")[1].strip().split()[0])
        elif "Oscillation angle" in line:
            oscillation_angle = float(line.split(":")[1].strip().split()[0])

print(f"Starting angle: {starting_angle} deg")
print(f"Ending angle: {ending_angle} deg")
print(f"Oscillation angle: {oscillation_angle} deg")

Starting angle: -71.69 deg
Ending angle: 71.54 deg
Oscillation angle: 0.1146 deg


In [6]:
#General search for other params:

def param_search(path, parameter):
    param = None
    with open(path) as file:
        for line in file:
            if parameter in line:
                param = float(line.split(":")[1].strip().split()[0])
    return param


In [13]:
#Setting the tilt angle for each frame to be [starting_angle + {frame index -1}*oscillation_angle]

with open('fish_cRED_manualinput.txt', 'r') as file:
    lines = file.readlines()

modified_lines = []
in_imagelist = False

for line in lines:
    partition = line.split()
    if "imagelist" in line:
        in_imagelist = True
    if in_imagelist:    
        if len(partition) == 3:
            frame_index = int(partition[0].split('/')[-1].split('.')[0])
            partition[1] = f"{starting_angle + ((frame_index -1)*oscillation_angle):.2f}"
            frame_index += 1
        modified_lines.append(' '.join(partition) + '\n')
    else:
        modified_lines.append(line)

with open('fish_cRED_manualinput.txt', 'w') as file:
    file.write(''.join(modified_lines))



In [1]:
#check if the last tilt value matches ending_angle
with open('fish_cRED_manualinput.txt', 'r') as file:
    lines = file.readlines()

endlist_index = lines.index('endimagelist\n')

end_frame = lines[endlist_index -1]

end_frame_angle = end_frame.split()[1]

In [7]:
abs(float(ending_angle) - float(end_frame_angle)) <= 0.1

True